# Connecting database

In [1]:
import ibm_db

In [2]:
#set database credentials
dsn_hostname = "dashdb-txn-sbox-yp-dal09-11.services.dal.bluemix.net" 
dsn_uid = "userid"        
dsn_pwd = "password"      

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            
dsn_port = "50000"               
dsn_protocol = "TCPIP"     

In [3]:
#Create the DB2 database connection
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

In [ ]:
#print the connection string to check correct values are specified
print(dsn)

In [ ]:
try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )

In [6]:
#Retrieve Metadata for the Database Server
server = ibm_db.server_info(conn)

print ("DBMS_NAME: ", server.DBMS_NAME)
print ("DBMS_VER:  ", server.DBMS_VER)
print ("DB_NAME:   ", server.DB_NAME)

DBMS_NAME:  DB2/LINUXX8664
DBMS_VER:   11.01.0404
DB_NAME:    BLUDB


In [7]:
#Retrieve Metadata for the Database Client / Driver
client = ibm_db.client_info(conn)

print ("DRIVER_NAME:          ", client.DRIVER_NAME) 
print ("DRIVER_VER:           ", client.DRIVER_VER)
print ("DATA_SOURCE_NAME:     ", client.DATA_SOURCE_NAME)
print ("DRIVER_ODBC_VER:      ", client.DRIVER_ODBC_VER)
print ("ODBC_VER:             ", client.ODBC_VER)
print ("ODBC_SQL_CONFORMANCE: ", client.ODBC_SQL_CONFORMANCE)
print ("APPL_CODEPAGE:        ", client.APPL_CODEPAGE)
print ("CONN_CODEPAGE:        ", client.CONN_CODEPAGE)

DRIVER_NAME:           libdb2.a
DRIVER_VER:            11.05.0400
DATA_SOURCE_NAME:      BLUDB
DRIVER_ODBC_VER:       03.51
ODBC_VER:              03.01.0000
ODBC_SQL_CONFORMANCE:  EXTENDED
APPL_CODEPAGE:         1208
CONN_CODEPAGE:         1208


In [8]:
#closing connection 
#ibm_db.close(conn)

# Creat table, insert and query data on DBMS by Python

In [9]:
#create table Trucks in Jupyter notebook by Python
stmt = ibm_db.exec_immediate(conn,"DROP TABLE Trucks;")
stmt = ibm_db.exec_immediate(conn,"CREATE TABLE Trucks(serial_no VARCHAR(20) PRIMARY KEY NOT NULL,model VARCHAR(20) NOT NULL,manufacturer VARCHAR(20) NOT NULL,engine_size VARCHAR(20) NOT NULL,truck_classn VARCHAR(20) NOT NULL);")

In [10]:
#add values
stmt = ibm_db.exec_immediate(conn,"INSERT INTO Trucks VALUES('A1234','Lonestar','International Trucks','Cummins ISX15','Class 8'),('B5432','Volvo VN','Volvo Trucks','Volvo D11','Heavy Duty Class 8'),('C5674','Kenworth W900','Kenworth Truck Co','Caterpillar C9','Class 8');")

In [11]:
#query data
stmt = ibm_db.exec_immediate(conn,"SELECT * FROM Trucks")
ibm_db.fetch_both(stmt)

{'SERIAL_NO': 'A1234',
 0: 'A1234',
 'MODEL': 'Lonestar',
 1: 'Lonestar',
 'MANUFACTURER': 'International Trucks',
 2: 'International Trucks',
 'ENGINE_SIZE': 'Cummins ISX15',
 3: 'Cummins ISX15',
 'TRUCK_CLASSN': 'Class 8',
 4: 'Class 8'}

In [12]:
#CREATE TABLE INSTRUCTOR

#drop table instructor if exists
dropStmt = ibm_db.exec_immediate(conn,"DROP TABLE Instructor;")

Stmt2 = ibm_db.exec_immediate(conn,"CREATE TABLE Instructor(id INT PRIMARY KEY NOT NULL,fname VARCHAR(20), lname VARCHAR(20), city VARCHAR(20), ccode CHARACTER(2));")

In [13]:
#insert data into Instructor
Stmt2 =ibm_db.exec_immediate(conn, "INSERT INTO Instructor VALUES(1,'Rav','Ahuja','TORONTO','CA'),(2,'Raul','Chong','Markham','CA'),(3,'Hima','Vasudevan','Chicago','US');")

In [14]:
#select data into Instructor
stmt2 =ibm_db.exec_immediate(conn, "SELECT * from Instructor;")
ibm_db.fetch_both(stmt2)

{'ID': 1,
 0: 1,
 'FNAME': 'Rav',
 1: 'Rav',
 'LNAME': 'Ahuja',
 2: 'Ahuja',
 'CITY': 'TORONTO',
 3: 'TORONTO',
 'CCODE': 'CA',
 4: 'CA'}

In [15]:
#fetch the dictionary for the first row only
stmt2 =ibm_db.exec_immediate(conn, "SELECT * from Instructor;")
ibm_db.fetch_both(stmt2)

{'ID': 1,
 0: 1,
 'FNAME': 'Rav',
 1: 'Rav',
 'LNAME': 'Ahuja',
 2: 'Ahuja',
 'CITY': 'TORONTO',
 3: 'TORONTO',
 'CCODE': 'CA',
 4: 'CA'}

In [16]:
#fetch the rest of the rows and print the id and fname for those rows
while ibm_db.fetch_row(stmt2) != False:
    print ("ID:", ibm_db.result(stmt2,0), "FNAME:", ibm_db.result(stmt2,'FNAME'))

ID: 2 FNAME: Raul
ID: 3 FNAME: Hima


In [17]:
#update Rav's city to MOOSETOWN
stmt2 =ibm_db.exec_immediate(conn, "UPDATE Instructor Set city='MOOSETOWN' where fname='Rav';")
stmt2 =ibm_db.exec_immediate(conn, "SELECT * from Instructor;")
ibm_db.fetch_both(stmt2)

{'ID': 1,
 0: 1,
 'FNAME': 'Rav',
 1: 'Rav',
 'LNAME': 'Ahuja',
 2: 'Ahuja',
 'CITY': 'MOOSETOWN',
 3: 'MOOSETOWN',
 'CCODE': 'CA',
 4: 'CA'}

# Retrieve data into Pandas

In [18]:
import pandas
import ibm_db_dbi

In [19]:
#connection for pandas
pconn=ibm_db_dbi.Connection(conn)

In [20]:
#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select * from INSTRUCTOR"

In [21]:
#retrieve the query results into a pandas dataframe
pdf = pandas.read_sql(selectQuery, pconn)

In [22]:
#print just the LNAME for first row in the pandas data frame
pdf.LNAME[0]

'Ahuja'

In [23]:
#print the entire data frame
pdf

,ID,FNAME,LNAME,CITY,CCODE
0,1,Rav,Ahuja,MOOSETOWN,CA
1,2,Raul,Chong,Markham,CA
2,3,Hima,Vasudevan,Chicago,US


In [24]:
pdf.shape

(3, 5)

In [25]:
ibm_db.close(conn)

True

# SQL Magic

In [26]:
#load the ipython-sql extension
%load_ext sql

In [27]:
#connect to DB2
%sql ibm_db_sa://username:password@dashdb-txn-sbox-yp-dal09-11.services.dal.bluemix.net:50000/BLUDB

In [28]:
%%sql
DROP TABLE INTERNATIONAL_STUDENT_TEST_SCORES;
CREATE TABLE INTERNATIONAL_STUDENT_TEST_SCORES (
    country VARCHAR(50),
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    test_score INT
    );
    INSERT INTO INTERNATIONAL_STUDENT_TEST_SCORES (country, first_name, last_name, test_score)
    VALUES
    ('United States', 'Marshall', 'Bernadot', 54),
    ('Ghana', 'Celinda', 'Malkin', 51),
    ('Ukraine', 'Guillermo', 'Furze', 53),
    ('Greece', 'Aharon', 'Tunnow', 48),
    ('Russia', 'Bail', 'Goodwin', 46),
    ('Poland', 'Cole', 'Winteringham', 49),
    ('Sweden', 'Emlyn', 'Erricker', 55),
    ('Russia', 'Cathee', 'Sivewright', 49),
    ('China', 'Barny', 'Ingerson', 57),
    ('Uganda', 'Sharla', 'Papaccio', 55),
    ('China', 'Stella', 'Youens', 51),
    ('Poland', 'Julio', 'Buesden', 48),
    ('United States', 'Tiffie', 'Cosely', 58),
    ('Poland', 'Auroora', 'Stiffell', 45),
    ('China', 'Clarita', 'Huet', 52),
    ('Poland', 'Shannon', 'Goulden', 45),
    ('Philippines', 'Emylee', 'Privost', 50),
    ('France', 'Madelina', 'Burk', 49),
    ('China', 'Saunderson', 'Root', 58),
    ('Indonesia', 'Bo', 'Waring', 55),
    ('China', 'Hollis', 'Domotor', 45),
    ('Russia', 'Robbie', 'Collip', 46),
    ('Philippines', 'Davon', 'Donisi', 46),
    ('China', 'Cristabel', 'Radeliffe', 48),
    ('China', 'Wallis', 'Bartleet', 58),
    ('Moldova', 'Arleen', 'Stailey', 38),
    ('Ireland', 'Mendel', 'Grumble', 58),
    ('China', 'Sallyann', 'Exley', 51),
    ('Mexico', 'Kain', 'Swaite', 46),
    ('Indonesia', 'Alonso', 'Bulteel', 45),
    ('Armenia', 'Anatol', 'Tankus', 51),
    ('Indonesia', 'Coralyn', 'Dawkins', 48),
    ('China', 'Deanne', 'Edwinson', 45),
    ('China', 'Georgiana', 'Epple', 51),
    ('Portugal', 'Bartlet', 'Breese', 56),
    ('Azerbaijan', 'Idalina', 'Lukash', 50),
    ('France', 'Livvie', 'Flory', 54),
    ('Malaysia', 'Nonie', 'Borit', 48),
    ('Indonesia', 'Clio', 'Mugg', 47),
    ('Brazil', 'Westley', 'Measor', 48),
    ('Philippines', 'Katrinka', 'Sibbert', 51),
    ('Poland', 'Valentia', 'Mounch', 50),
    ('Norway', 'Sheilah', 'Hedditch', 53),
    ('Papua New Guinea', 'Itch', 'Jubb', 50),
    ('Latvia', 'Stesha', 'Garnson', 53),
    ('Canada', 'Cristionna', 'Wadmore', 46),
    ('China', 'Lianna', 'Gatward', 43),
    ('Guatemala', 'Tanney', 'Vials', 48),
    ('France', 'Alma', 'Zavittieri', 44),
    ('China', 'Alvira', 'Tamas', 50),
    ('United States', 'Shanon', 'Peres', 45),
    ('Sweden', 'Maisey', 'Lynas', 53),
    ('Indonesia', 'Kip', 'Hothersall', 46),
    ('China', 'Cash', 'Landis', 48),
    ('Panama', 'Kennith', 'Digance', 45),
    ('China', 'Ulberto', 'Riggeard', 48),
    ('Switzerland', 'Judy', 'Gilligan', 49),
    ('Philippines', 'Tod', 'Trevaskus', 52),
    ('Brazil', 'Herold', 'Heggs', 44),
    ('Latvia', 'Verney', 'Note', 50),
    ('Poland', 'Temp', 'Ribey', 50),
    ('China', 'Conroy', 'Egdal', 48),
    ('Japan', 'Gabie', 'Alessandone', 47),
    ('Ukraine', 'Devlen', 'Chaperlin', 54),
    ('France', 'Babbette', 'Turner', 51),
    ('Czech Republic', 'Virgil', 'Scotney', 52),
    ('Tajikistan', 'Zorina', 'Bedow', 49),
    ('China', 'Aidan', 'Rudeyeard', 50),
    ('Ireland', 'Saunder', 'MacLice', 48),
    ('France', 'Waly', 'Brunstan', 53),
    ('China', 'Gisele', 'Enns', 52),
    ('Peru', 'Mina', 'Winchester', 48),
    ('Japan', 'Torie', 'MacShirrie', 50),
    ('Russia', 'Benjamen', 'Kenford', 51),
    ('China', 'Etan', 'Burn', 53),
    ('Russia', 'Merralee', 'Chaperlin', 38),
    ('Indonesia', 'Lanny', 'Malam', 49),
    ('Canada', 'Wilhelm', 'Deeprose', 54),
    ('Czech Republic', 'Lari', 'Hillhouse', 48),
    ('China', 'Ossie', 'Woodley', 52),
    ('Macedonia', 'April', 'Tyer', 50),
    ('Vietnam', 'Madelon', 'Dansey', 53),
    ('Ukraine', 'Korella', 'McNamee', 52),
    ('Jamaica', 'Linnea', 'Cannam', 43),
    ('China', 'Mart', 'Coling', 52),
    ('Indonesia', 'Marna', 'Causbey', 47),
    ('China', 'Berni', 'Daintier', 55),
    ('Poland', 'Cynthia', 'Hassell', 49),
    ('Canada', 'Carma', 'Schule', 49),
    ('Indonesia', 'Malia', 'Blight', 48),
    ('China', 'Paulo', 'Seivertsen', 47),
    ('Niger', 'Kaylee', 'Hearley', 54),
    ('Japan', 'Maure', 'Jandak', 46),
    ('Argentina', 'Foss', 'Feavers', 45),
    ('Venezuela', 'Ron', 'Leggitt', 60),
    ('Russia', 'Flint', 'Gokes', 40),
    ('China', 'Linet', 'Conelly', 52),
    ('Philippines', 'Nikolas', 'Birtwell', 57),
    ('Australia', 'Eduard', 'Leipelt', 53)

 * ibm_db_sa://zgm80331:***@dashdb-txn-sbox-yp-dal09-11.services.dal.bluemix.net:50000/BLUDB
Done.
Done.
99 rows affected.


[]

## Using Python Variables in SQL Statements

In [29]:
country = "Canada"
%sql select * from INTERNATIONAL_STUDENT_TEST_SCORES where country = :country

 * ibm_db_sa://zgm80331:***@dashdb-txn-sbox-yp-dal09-11.services.dal.bluemix.net:50000/BLUDB
Done.


country,first_name,last_name,test_score
Canada,Cristionna,Wadmore,46
Canada,Wilhelm,Deeprose,54
Canada,Carma,Schule,49


 ## Assigning the Results of Queries to Python Variables

In [30]:
test_score_distribution = %sql SELECT test_score as "Test Score", count(*) as "Frequency" from INTERNATIONAL_STUDENT_TEST_SCORES GROUP BY test_score;
test_score_distribution

 * ibm_db_sa://zgm80331:***@dashdb-txn-sbox-yp-dal09-11.services.dal.bluemix.net:50000/BLUDB
Done.


Test Score,Frequency
38,2
40,1
43,2
44,2
45,8
46,7
47,4
48,14
49,8
50,10


## Converting Query Results to DataFrames

In [31]:
test_score = pandas.DataFrame(data = test_score_distribution)
test_score 

,0,1
0,38,2
1,40,1
2,43,2
3,44,2
4,45,8
5,46,7
6,47,4
7,48,14
8,49,8
9,50,10


In [32]:
print(type(test_score_distribution))

<class 'sql.run.ResultSet'>


In [33]:
%%sql
SELECT country, first_name, last_name, test_score FROM INTERNATIONAL_STUDENT_TEST_SCORES;

 * ibm_db_sa://zgm80331:***@dashdb-txn-sbox-yp-dal09-11.services.dal.bluemix.net:50000/BLUDB
Done.


country,first_name,last_name,test_score
United States,Marshall,Bernadot,54
Ghana,Celinda,Malkin,51
Ukraine,Guillermo,Furze,53
Greece,Aharon,Tunnow,48
Russia,Bail,Goodwin,46
Poland,Cole,Winteringham,49
Sweden,Emlyn,Erricker,55
Russia,Cathee,Sivewright,49
China,Barny,Ingerson,57
Uganda,Sharla,Papaccio,55
